### 1. 단어장 크기별 ML 성능비교표

![단어장 크기별 ML성능 비교표](./단어장크기별ML성능비교표.png)

### 2. ML과 딥러닝 성능비교표

![ML과딥러닝 성능비교표](./ML과딥러닝성능비교표.png)

In [1]:
from tensorflow.keras.datasets import reuters
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

### 데이터준비
- index -> text
- DTM, TF-idf 학습 데이터 준비
- Word2Vector 학습 데이터 준비

In [2]:
# 이부분에있는 num_words를 5000과 다른방법으로 바꿔보세요~

(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=None, test_split=0.2)

In [3]:
# 원본 뉴스 데이터로 복원해보기
word_index = reuters.get_word_index(path="reuters_word_index.json")

In [4]:
# 정수로부터 단어를 얻을 수 있는 index_word
index_to_word = { index+3 : word for word, index in word_index.items() }

# index_to_word에 숫자 0은 <pad>, 숫자 1은 <sos>, 숫자 2는 <unk>를 넣어줍니다.
for index, token in enumerate(("<pad>", "<sos>", "<unk>")):
    index_to_word[index]=token

In [5]:
# 전체 훈련용 뉴스 데이터와 전체 테스트용 뉴스 데이터를 텍스트 데이터로 변환해 보겠습니다.

# 전체 훈련용 뉴스 데이터를 텍스트 데이터로 변환
decoded = []
for i in range(len(x_train)):
    t = ' '.join([index_to_word[index] for index in x_train[i]])
    decoded.append(t)
    
x_train = decoded
print(len(x_train))

# 전체 테스트용 뉴스 데이터를 텍스트 데이터로 변환
decoded = []
for i in range(len(x_test)):
    t = ' '.join([index_to_word[index] for index in x_test[i]])
    decoded.append(t)
    
x_test = decoded
print(len(x_test))

8982
2246


In [6]:
# 벡터화 DTM, TF-idf 방법
dtmvector = CountVectorizer()
tfidf_transformer = TfidfTransformer()

x_train_dtm = dtmvector.fit_transform(x_train)
x_test_dtm= dtmvector.transform(x_test)

x_train_tfidf = tfidf_transformer.fit_transform(x_train_dtm)
x_test_tfidf = tfidf_transformer.transform(x_test_dtm)

In [7]:
# 벡터화 W2V방법
from gensim.models import Word2Vec

# 우선 문장을 토큰화 시킵시다 띄어쓰기 기반으로 해볼게요! -> # 위에서 DTM만들때는 왜 안해줬냐! -> CountVectorizer에서 띄어쓰기 기반 토큰화가 내장되있음
x_train_tokenized = [sentence.split() for sentence in x_train]
x_test_tokenized = [sentence.split() for sentence in x_test]

# vector사이즈를 늘리거나 줄여보세요 아마 512 가장많이쓰이는 방식
model = Word2Vec(sentences = x_train_tokenized, vector_size = 256, window = 5, min_count = 5, workers = 4, sg = 0)
print("모델 학습 완료!")

모델 학습 완료!


In [8]:
# W2V이 잘되었는지 확인
model_result = model.wv.most_similar('man')
print(model_result)

[('harbour', 0.8768444657325745), ('halmi', 0.876131534576416), ('jim', 0.8596214056015015), ('monopoly', 0.8559353351593018), ('shelf', 0.8543671369552612), ('okla', 0.8474838137626648), ('puerto', 0.8456472754478455), ('alcan', 0.845412015914917), ('missouri', 0.8443841934204102), ('alabama', 0.8442700505256653)]


In [9]:
# 학습된 Word2Vec 모델
w2v_model = model

# 각 문장을 벡터화 시키는 코드
def vectorize_sentence(sentence, model, max_len):
    vecs = []
    for word in sentence:
        if word in model.wv:
            vecs.append(model.wv[word])
        else:
            vecs.append(np.zeros(model.vector_size))
    # Padding
    if len(vecs) < max_len:
        vecs += [np.zeros(model.vector_size)] * (max_len - len(vecs))
    else:
        vecs = vecs[:max_len]
    return np.array(vecs)


# 최대 문장길이를 잘 잡아주세요
x_train_w2v = np.array([vectorize_sentence(s, w2v_model, max_len=100) for s in x_train_tokenized])
x_test_w2v = np.array([vectorize_sentence(s, w2v_model, max_len=100) for s in x_test_tokenized])

In [10]:
x_train_w2v.shape

(8982, 100, 256)

In [11]:
x_test_w2v.shape

(2246, 100, 256)

### 모델 정의 및 실험

In [12]:
# 결정트리 분류모델 학습
from sklearn.tree import DecisionTreeClassifier

dt_clf = DecisionTreeClassifier(max_depth=10, random_state=0)
dt_clf.fit(x_train_tfidf, y_train)

DecisionTreeClassifier(max_depth=10, random_state=0)

In [13]:
# 예측
y_pred = dt_clf.predict(x_test_tfidf)

# 평가지표
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"✅ Accuracy : {acc:.4f}")
print(f"✅ F1-score : {f1:.4f}")

✅ Accuracy : 0.6211
✅ F1-score : 0.5769


In [14]:
# 데이터를 단어단위에서 문장단위로 바꿔줘야합니다.. ML은 2차원데이터만 받을수있기때문
# 문장에 대해서 토큰들의 벡터를 평균을 취해줍니다.

# Word2Vec 임베딩 시퀀스: (8982, 100, 256)
x_w2v_seq_train = x_train_w2v
x_w2v_seq_test = x_test_w2v
# 평균 풀링 → (8982, 256)
x_w2v_avg_train = np.mean(x_w2v_seq_train, axis=1)
x_w2v_avg_test = np.mean(x_w2v_seq_test, axis=1)

print(x_w2v_avg_train.shape)  # (8982, 256)

(8982, 256)


In [15]:
# Word2Vec 데이터로 결정트리 분류 모델 학습하기
from sklearn.tree import DecisionTreeClassifier

dt_clf = DecisionTreeClassifier(max_depth=10, random_state=0)
dt_clf.fit(x_w2v_avg_train, y_train)

DecisionTreeClassifier(max_depth=10, random_state=0)

In [16]:
# 예측
y_pred = dt_clf.predict(x_w2v_avg_test)

# 평가 지표
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"✅ Accuracy : {acc:.4f}")
print(f"✅ F1-score : {f1:.4f}")

✅ Accuracy : 0.6443
✅ F1-score : 0.6374


### Dense NN 딥러닝 모델

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, LSTM, Dense, Dropout


dense_model = Sequential([
    Flatten(input_shape=(100, 256)),  # (seq_len, embedding_dim)
    Dense(512, activation='relu'),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(46, activation='softmax')   # 클래스 수에 맞게 조정 46개로 맞춰주세요!
])

dense_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
dense_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 25600)             0         
_________________________________________________________________
dense (Dense)                (None, 512)               13107712  
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               65664     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 46)                5934      
Total params: 13,179,310
Trainable params: 13,179,310
Non-trainable params: 0
____________________________________________

In [18]:
# 시간이 좀 걸립니다! 한 20분정도..

# Word2Vec 벡터화된 학습 데이터
dense_model.fit(x_train_w2v, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
225/225 [==============================] - 3s 8ms/step - loss: 1.8857 - accuracy: 0.5819 - val_loss: 1.4623 - val_accuracy: 0.6728
Epoch 2/10
225/225 [==============================] - 1s 6ms/step - loss: 1.3457 - accuracy: 0.6788 - val_loss: 1.3719 - val_accuracy: 0.6945
Epoch 3/10
225/225 [==============================] - 1s 6ms/step - loss: 1.0807 - accuracy: 0.7383 - val_loss: 1.3859 - val_accuracy: 0.6789
Epoch 4/10
225/225 [==============================] - 1s 6ms/step - loss: 0.9137 - accuracy: 0.7822 - val_loss: 1.4035 - val_accuracy: 0.6856
Epoch 5/10
225/225 [==============================] - 1s 6ms/step - loss: 0.7423 - accuracy: 0.8129 - val_loss: 1.4242 - val_accuracy: 0.6861
Epoch 6/10
225/225 [==============================] - 1s 6ms/step - loss: 0.6658 - accuracy: 0.8373 - val_loss: 1.4500 - val_accuracy: 0.6967
Epoch 7/10
225/225 [==============================] - 1s 6ms/step - loss: 0.5796 - accuracy: 0.8575 - val_loss: 1.5799 - val_accuracy: 0.6811
Epoch 

In [19]:
y_pred_proba = dense_model.predict(x_test_w2v)
y_pred = np.argmax(y_pred_proba, axis=1)

acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"✅ Accuracy: {acc:.4f}")
print(f"✅ F1-score: {f1:.4f}")

✅ Accuracy: 0.6906
✅ F1-score: 0.6730


### RNN 딥러닝 모델

In [20]:
# rnn 시계열 특징 데이터 특화 모델

rnn_model = Sequential([
    LSTM(128, input_shape=(100, 256)),  # (seq_len, embedding_dim)
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(46, activation='softmax')   # 클래스 수에 맞게 조정 46개로 맞춰주세요~
])

rnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
rnn_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               197120    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 46)                2990      
Total params: 208,366
Trainable params: 208,366
Non-trainable params: 0
_________________________________________________________________


In [21]:
# 시간이 좀 걸립니다! 한 20분정도
rnn_model.fit(x_train_w2v, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
225/225 [==============================] - 5s 12ms/step - loss: 2.1990 - accuracy: 0.4589 - val_loss: 1.7804 - val_accuracy: 0.5615
Epoch 2/10
225/225 [==============================] - 2s 8ms/step - loss: 1.6835 - accuracy: 0.5904 - val_loss: 1.5447 - val_accuracy: 0.6216
Epoch 3/10
225/225 [==============================] - 2s 8ms/step - loss: 1.5066 - accuracy: 0.6316 - val_loss: 1.3851 - val_accuracy: 0.6578
Epoch 4/10
225/225 [==============================] - 2s 8ms/step - loss: 1.4239 - accuracy: 0.6519 - val_loss: 1.2859 - val_accuracy: 0.6850
Epoch 5/10
225/225 [==============================] - 2s 8ms/step - loss: 1.2814 - accuracy: 0.6877 - val_loss: 1.1840 - val_accuracy: 0.7168
Epoch 6/10
225/225 [==============================] - 2s 8ms/step - loss: 1.2053 - accuracy: 0.7061 - val_loss: 1.1600 - val_accuracy: 0.7223
Epoch 7/10
225/225 [==============================] - 2s 8ms/step - loss: 1.1656 - accuracy: 0.7116 - val_loss: 1.1606 - val_accuracy: 0.7268
Epoch

In [22]:
y_pred_proba = rnn_model.predict(x_test_w2v)
y_pred = np.argmax(y_pred_proba, axis=1)

acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"✅ Accuracy: {acc:.4f}")
print(f"✅ F1-score: {f1:.4f}")

✅ Accuracy: 0.7182
✅ F1-score: 0.6853
